In [1]:
from despliegue.loaders import *
from despliegue.nodos import *
from despliegue.contenedores import *
from despliegue.solvers import *
import pandas as pd
from os import path

Se definen las rutas de los datos, y también el `DataFrame` de la Red Móvil, ya que no posee la vacancia, y hay que asignarla a mano.

In [2]:
PATH_DATA = path.join("./data/")
PATH_CLIENTES = path.join(PATH_DATA, "Direcciones_Colina.xlsx")
PATH_FO = path.join(PATH_DATA, "CTO_Colina.xlsx")
PATH_RM = path.join(PATH_DATA, "SITIOS 4G TDD.xlsx")

In [3]:
df_clientes = pd.read_excel(PATH_CLIENTES)
df_clientes = df_clientes[df_clientes.gl_lat_OK <= -1]  # limpiamos aquellos que sean de lat-lon=0
df_clientes.describe()

,pcm_area_tel,gl_lat_OK,gl_lon_OK
count,2.750000e+02,275.000000,275.000000
mean,2.286022e+08,-33.200679,-70.672530
std,1.034212e+06,0.013769,0.011696
min,2.240048e+08,-33.301086,-70.723406
25%,2.284422e+08,-33.207438,-70.679538
50%,2.284444e+08,-33.202714,-70.673384
75%,2.284477e+08,-33.192004,-70.666531
max,2.329671e+08,-33.173324,-70.632150


In [4]:
df_clientes.at[1, "zona"] = 8
df_clientes

,pcm_area_tel,zona,gl_lat_OK,gl_lon_OK,pcm_comuna_inst
0,228440583,RM,-33.205060,-70.671429,COLINA
1,228603381,8,-33.181246,-70.659071,COLINA
2,228602696,RM,-33.194427,-70.667572,COLINA
3,228449326,RM,-33.203642,-70.673384,COLINA
4,228440972,RM,-33.181033,-70.646756,COLINA
...,...,...,...,...,...
301,228445321,RM,-33.202984,-70.677652,COLINA
302,228446910,RM,-33.196779,-70.670552,COLINA
303,228445693,RM,-33.202952,-70.672505,COLINA
305,228442430,RM,-33.208543,-70.682372,COLINA


In [5]:
df_fo = pd.read_excel(PATH_FO)
df_fo.describe()

,id_pto_ftth,latitud_ok,longitud_ok,eqpt_capacity,eqpt_vg_qty,oc_ds,porcentaje_ocup_caja
count,3.484000e+03,3484.000000,3484.000000,3484.000000,3484.00000,3484.00000,3484.000000
mean,2.476223e+06,-33.202056,-70.673801,7.756889,5.74225,2.00287,23.659374
std,4.067262e+05,0.013274,0.009063,0.839911,1.56150,1.40341,17.531295
min,1.758698e+06,-33.222792,-70.711280,1.000000,1.00000,0.00000,0.000000
25%,2.120851e+06,-33.214408,-70.681072,8.000000,5.00000,1.00000,12.500000
50%,2.478504e+06,-33.203207,-70.673720,8.000000,6.00000,2.00000,25.000000
75%,2.828275e+06,-33.190197,-70.667940,8.000000,7.00000,3.00000,37.500000
max,3.177018e+06,-33.176260,-70.649690,9.000000,8.00000,5.00000,57.140000


In [6]:
df_rm = pd.read_excel(PATH_RM)
df_rm["vacancia"] = 30
df_rm.describe()

,LATITUD,LONGITUD,vacancia
count,99.000000,99.000000,99.0
mean,-33.504739,-70.714568,30.0
std,0.116812,0.102712,0.0
min,-33.810990,-71.225640,30.0
25%,-33.576260,-70.746720,30.0
50%,-33.509860,-70.695800,30.0
75%,-33.409805,-70.664650,30.0
max,-33.205890,-70.537380,30.0


Creamos las instancias de conexión a la base de datos. Observar que para `rm_db` le entregamos el `DataFrame` directamente.

In [7]:
cliente_db = ClienteDB(df=df_clientes, col_names=["pcm_area_tel", "gl_lat_OK", "gl_lon_OK"])
fo_db = OfertaDB(df=df_fo, col_names=["id_pto_ftth", "latitud_ok", "longitud_ok", "eqpt_vg_qty"])
rm_db = OfertaDB(df=df_rm)

Creamos los conjuntos de nodos

In [8]:
nodos_oferta = NodosOferta(fo_db, rm_db)
nodos_demanda = NodosDemanda(cliente_db)

In [9]:
# modelo1 = Solver1(nodos_oferta, nodos_demanda, verbose=Tru
# modelo.construir_modelo()
# modelo.resolver()
# modelo.variables

In [10]:
# modelo2 = Solver2(nodos_oferta, nodos_demanda, verbose=True)
# modelo2.construir_modelo()
# modelo2.resolver()
# modelo2.variables

In [16]:
modelo3 = Solver3(nodos_oferta, nodos_demanda, a=100, b=1000, c_fo=150, c_rm=600, verbose=True)
modelo3.construir_modelo()
modelo3.resolver()

Construyendo modelo...
Empezando a resolver...
Estado: Optimal
x_(1013,_115) = 1.0
x_(1013,_260) = 1.0
x_(1031,_196) = 1.0
x_(1040,_30) = 1.0
x_(1098,_161) = 1.0
x_(1098,_188) = 1.0
x_(1123,_273) = 1.0
x_(1176,_102) = 1.0
x_(1234,_75) = 1.0
x_(1314,_244) = 1.0
x_(1322,_148) = 1.0
x_(1338,_185) = 1.0
x_(1343,_38) = 1.0
x_(1351,_117) = 1.0
x_(1364,_77) = 1.0
x_(1375,_209) = 1.0
x_(142,_262) = 1.0
x_(1458,_53) = 1.0
x_(1490,_200) = 1.0
x_(1508,_261) = 1.0
x_(1542,_152) = 1.0
x_(1551,_135) = 1.0
x_(1558,_57) = 1.0
x_(1568,_121) = 1.0
x_(1584,_201) = 1.0
x_(1584,_259) = 1.0
x_(1584,_271) = 1.0
x_(1584,_69) = 1.0
x_(1605,_190) = 1.0
x_(1640,_257) = 1.0
x_(1655,_92) = 1.0
x_(1668,_213) = 1.0
x_(1697,_131) = 1.0
x_(1725,_218) = 1.0
x_(173,_109) = 1.0
x_(1746,_47) = 1.0
x_(1770,_217) = 1.0
x_(1784,_1) = 1.0
x_(179,_122) = 1.0
x_(1803,_16) = 1.0
x_(1807,_253) = 1.0
x_(1807,_76) = 1.0
x_(1814,_140) = 1.0
x_(1837,_48) = 1.0
x_(1868,_111) = 1.0
x_(1868,_264) = 1.0
x_(1868,_274) = 1.0
x_(1873,_123) 

In [19]:

df = pd.DataFrame(columns=["cl_id", "cl_lat", "cl_lon", "cate_oferta", "oferta_id"])

for j in modelo3.demanda.indice:
    for i in modelo3.demanda[j].vecinos:
        var = modelo3.x[i, j]
        if var.varValue != 0:
            df.at[j, "cl_id"] = int(modelo3.demanda[j].id)
            df.at[j, "cl_lat"] = modelo3.demanda[j].lat
            df.at[j, "cl_lon"] = modelo3.demanda[j].lon
            df.at[j, "cate_oferta"] = modelo3.oferta[i].cate
            df.at[j, "oferta_id"] = modelo3.oferta[i].id
            
df

,cl_id,cl_lat,cl_lon,cate_oferta,oferta_id
0,228440583,-33.20506,-70.671429,FO,2716184.0
1,228603381,-33.181246,-70.659071,FO,2498715.0
2,228602696,-33.194427,-70.667572,FO,2656793.0
3,228449326,-33.203642,-70.673384,FO,2699989.0
4,228440972,-33.181033,-70.646756,Sumidero,
...,...,...,...,...,...
270,228445321,-33.202984,-70.677652,RM,CLINFD13
271,228446910,-33.196779,-70.670552,FO,2414791.0
272,228445693,-33.202952,-70.672505,FO,2986891.0
273,228442430,-33.208543,-70.682372,FO,2229986.0


In [20]:
modelo3.save(path.join(PATH_DATA, "resultado.xlsx"))

In [ ]:
len(modelo3.variables), len(modelo3.x)